In [1]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn import svm
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [2]:
inlier_train = pd.read_csv('inlier.csv',header=None)
outlier_train =  pd.read_csv('outlier.csv',header=None)
#test =  pd.read_csv('test.csv',header=None)

In [3]:
y_inlier=np.zeros( (inlier_train.shape[0],1), dtype=np.int16 )
y_outlier=np.ones( (outlier_train.shape[0],1), dtype=np.int16 )

In [4]:
X= pd.concat([inlier_train,outlier_train], axis=0)

In [5]:
y=np.append(y_inlier,y_outlier)

In [6]:
# apply standard scaler to output from resnet50
ss = StandardScaler()
ss.fit(X)
x_train = ss.transform(X)
#test_x130 = ss.transform(test)#####小写der

In [7]:
outlier_per=outlier_train.shape[0]/(outlier_train.shape[0]+inlier_train.shape[0])

#  outlier= 0.07    小的x_test  是真的x_test
####小的x_test  是真的x_test
1.......疯狂换模型   ensemble

2.....加outlier     


In [33]:
#recall_score(y_true, y_pred, average='binary')
###evaluation：：       文档里只说了是   model的 recall 跟    precision     ....没说 特指是正例.....  关注整体的accuracy   or   outlier?
#F1    ##f1_score(y_true, y_pred, average='weighted')   
#算了 管他呢 用官方文档score   ROC(其实我觉得他说的是AUC。。。。。0.5 是扔骰子。。。。。precision@n        )

###最后还是决定加上pca   神特么也太慢了吧    
####但是加上pca就掉了 ......慢就慢吧.... 有一个能用就成......

# Take PCA to reduce feature space dimensionality for inlier and test
pca = PCA(n_components=500, whiten=True)
pca = pca.fit(inlier_train)
print('Explained variance percentage = %0.2f' % sum(pca.explained_variance_ratio_))
print('num_features：', pca.n_components_)
inlier_train = pca.transform(inlier_train)
test = pca.transform(test)

In [8]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import f1_score
from pyod.models.iforest import IForest
from pyod.utils.data import generate_data
from pyod.utils.data import get_color_codes
from pyod.utils.data import evaluate_print
from pyod.models.abod import ABOD
from pyod.models.auto_encoder import AutoEncoder
from pyod.models.abod import ABOD
from pyod.models.cblof import CBLOF
from pyod.models.feature_bagging import FeatureBagging
from pyod.models.hbos import HBOS
from pyod.models.iforest import IForest
from pyod.models.knn import KNN
from pyod.models.lof import LOF
from pyod.models.loci import LOCI
from pyod.models.mcd import MCD
from pyod.models.ocsvm import OCSVM
from pyod.models.pca import PCA
from pyod.models.sos import SOS
from __future__ import division
from __future__ import print_function

import os
import sys

# temporary solution for relative imports in case pyod is not installed
# if pyod is installed, no need to use the following line
sys.path.append(
    os.path.abspath(os.path.join(os.path.dirname("__file__"), '..')))

# supress warnings for clean output
import warnings

warnings.filterwarnings("ignore")
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import matplotlib.font_manager

Using TensorFlow backend.


In [9]:
X_train, X_test, y_train, y_test = train_test_split( x_train, y, test_size=0.3, random_state=42)

In [10]:
outliers_fraction = outlier_per  # percentage of outliers
clf_name = 'IForest'
clf = KNN(contamination=outliers_fraction)
clf.fit(X_train)
# get the prediction labels and outlier scores of the training data
y_train_pred = clf.labels_  # binary labels (0: inliers, 1: outliers)
y_train_scores = clf.decision_scores_  # raw outlier scores

# get the prediction on the test data
y_test_pred = clf.predict(X_test)  # outlier labels (0 or 1)
y_test_scores = clf.decision_function(X_test)  # outlier scores

# evaluate and print the results
print("\nOn Training Data:")
evaluate_print(clf_name, y_train, y_train_scores)
print("\nOn Test Data:")
evaluate_print(clf_name, y_test, y_test_scores)


On Training Data:
IForest ROC:0.9052, precision @ rank n:0.4588

On Test Data:
IForest ROC:0.9133, precision @ rank n:0.5522


In [12]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_test_pred)

array([[1375,   56],
       [  63,   71]], dtype=int64)

In [13]:
random_state = np.random.RandomState(42)
# Define nine outlier detection tools to be compared
classifiers = {
                # 'Angle-based Outlier Detector (ABOD)':
                #    ABOD(contamination=outliers_fraction),
               'Fast Angle-based Outlier Detector (FastABOD)':
                   ABOD(contamination=outliers_fraction),
               'Cluster-based Local Outlier Factor (CBLOF)':
                   CBLOF(contamination=outliers_fraction,
                         check_estimator=False, random_state=random_state),
               'Feature Bagging':
                   FeatureBagging(LOF(n_neighbors=35),
                                  contamination=outliers_fraction,
                                  check_estimator=False,
                                  random_state=random_state),
               'Histogram-base Outlier Detection (HBOS)': HBOS(
                   contamination=outliers_fraction),
               'Isolation Forest': IForest(contamination=outliers_fraction,
                                           random_state=random_state),
               'K Nearest Neighbors (KNN)': KNN(
                   contamination=outliers_fraction),
               'Average KNN': KNN(method='mean',
                                  contamination=outliers_fraction),
               'Median KNN': KNN(method='median',
                                 contamination=outliers_fraction),
               'Local Outlier Factor (LOF)':
                   LOF(n_neighbors=35, contamination=outliers_fraction),
               # 'Local Correlation Integral (LOCI)':
               #     LOCI(contamination=outliers_fraction),
              
               'One-class SVM (OCSVM)': OCSVM(contamination=outliers_fraction,
                                              random_state=random_state),
               'Principal Component Analysis (PCA)': PCA(
                   contamination=outliers_fraction, random_state=random_state),
                'Auto encoder': AutoEncoder(epochs=30, contamination=outliers_fraction, random_state=random_state)
               # 'Stochastic Outlier Selection (SOS)': SOS(
               #     contamination=outliers_fraction),
               }

In [14]:
from sklearn.externals import joblib

#  存一下model   mdzz

In [17]:
# Show all detectors
for i, clf in enumerate(classifiers.keys()):
    print('Model', i + 1, clf)

# Fit the models with the generated data and
# compare model performances

models={}
for i, (clf_name, clf) in enumerate(classifiers.items()):
    print()
    print(i + 1, 'fitting', clf_name)
    # fit the data and tag outliers
    clf.fit(X_train)
    # get the prediction labels and outlier scores of the training data
    y_train_pred = clf.labels_  # binary labels (0: inliers, 1: outliers)
    y_train_scores = clf.decision_scores_  # raw outlier scores

    # get the prediction on the test data
    y_test_pred = clf.predict(X_test)  # outlier labels (0 or 1)
    y_test_scores = clf.decision_function(X_test)  # outlier scores
    print(confusion_matrix(y_test,y_test_pred))
    models[clf_name]=clf
   # joblib.dump(clf, 'model_%s.pkl' % (clf_name))
    # evaluate and print the results
    print("\nOn Training Data:")
    evaluate_print(clf_name, y_train, y_train_scores)
    print("\nOn Test Data:")
    evaluate_print(clf_name, y_test, y_test_scores)

Model 1 Fast Angle-based Outlier Detector (FastABOD)
Model 2 K Nearest Neighbors (KNN)
Model 3 Median KNN
Model 4 One-class SVM (OCSVM)
Model 5 Cluster-based Local Outlier Factor (CBLOF)
Model 6 Average KNN
Model 7 Principal Component Analysis (PCA)
Model 8 Local Outlier Factor (LOF)
Model 9 Histogram-base Outlier Detection (HBOS)
Model 10 Isolation Forest
Model 11 Feature Bagging

1 fitting Fast Angle-based Outlier Detector (FastABOD)
[[1335   96]
 [ 102   32]]

On Training Data:
Fast Angle-based Outlier Detector (FastABOD) ROC:0.7227, precision @ rank n:0.2007

On Test Data:
Fast Angle-based Outlier Detector (FastABOD) ROC:0.7473, precision @ rank n:0.2463

2 fitting K Nearest Neighbors (KNN)
[[1375   56]
 [  63   71]]

On Training Data:
K Nearest Neighbors (KNN) ROC:0.9052, precision @ rank n:0.4588

On Test Data:
K Nearest Neighbors (KNN) ROC:0.9133, precision @ rank n:0.5522

3 fitting Median KNN
[[1373   58]
 [  67   67]]

On Training Data:
Median KNN ROC:0.8992, precision @ rank

In [18]:
full_test=pd.read_csv('test_full.csv',header=None)
submission=pd.DataFrame()
submission['ID']=full_test.iloc[:,-1]


In [19]:
X_testfull=full_test.drop(columns=2048)
#X

In [20]:
X_testfull=ss.transform(X_testfull)

In [ ]:
for i, (clf_name, clf) in enumerate(classifiers.items()):
    print()
    print(i + 1, 'predicting', clf_name)
    model= joblib.load('model_%s.pkl' % (clf_name))
    y_test_pred = model.predict(X_testfull) 
    y_test_scores = model.decision_function(X_testfull)
    submission[clf_name]=y_test_pred
submission.to_csv('submission.csv',index=False)


1 predicting Fast Angle-based Outlier Detector (FastABOD)

2 predicting K Nearest Neighbors (KNN)

3 predicting Median KNN

4 predicting One-class SVM (OCSVM)

5 predicting Cluster-based Local Outlier Factor (CBLOF)

6 predicting Average KNN

7 predicting Principal Component Analysis (PCA)

8 predicting Local Outlier Factor (LOF)


In [ ]:
submission.to_csv('submission.csv',index=False)

In [ ]:
outlier_per